In [2]:
import cv2
import numpy as np
from scipy.spatial import distance as dist
from skimage.filters import threshold_local

In [3]:
def order_pts(pts):
    rect=np.zeros((4,2),dtype="float32")
    s=pts.sum(axis=1)
    rect[0]=pts[np.argmin(s)]
    rect[2]=pts[np.argmax(s)]
    diff=np.diff(pts,axis=1)
    rect[1]=pts[np.argmin(diff)]
    rect[3]=pts[np.argmax(diff)]
    return rect

In [4]:
def four_point_transform(img,pts):
    rect=order_pts(pts)
    (tl,tr,br,bl)=rect
    width_A=np.sqrt(((br[0]-bl[0])**2)+((br[1]-bl[1])**2))
    width_B=np.sqrt(((tr[0]-tl[0])**2)+((tr[1]-tl[1])**2))
    max_width=max(int(width_A),int(width_B))
    height_A=np.sqrt(((tr[0]-br[0])**2)+((tr[1]-br[1])**2))
    height_B=np.sqrt(((tl[0]-bl[0])**2)+((tr[1]-bl[1])**2))
    max_height=max(int(height_A),int(height_B))
    dst=np.array([[0,0],[max_width-1,0],[max_width-1,max_width-1],[0,max_height-1]],dtype="float32")
    M=cv2.getPerspectiveTransform(rect,dst)
    warped=cv2.warpPerspective(img,M,(max_width,max_height))
    return warped

In [5]:
def img_resize(img,width=None,height=None,inter=cv2.INTER_AREA):
    dim=None
    (h,w)=img.shape[:2]
    if width is None and height is None:
        return
    if width is None:
        r=height/float(h)
        dim=(int(w*r),height)
    else:
        r=width/float(w)
        dim=(width,int(h*r))
    return cv2.resize(img,dim,interpolation=inter)

In [20]:
img=cv2.imread('justpaper.jpg')

ratio=img.shape[0]/500.0
orig=img.copy()
img=img_resize(img,height=500)

blur=cv2.GaussianBlur(img,(5,5),0)
edge=cv2.Canny(blur,75,200)


cnts,hier=cv2.findContours(edge.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
cnts=sorted(cnts,key=cv2.contourArea,reverse=True)[:5]
for c in cnts:
    #print(c)
    peri=cv2.arcLength(c,True)
    approx=cv2.approxPolyDP(c,0.02*peri,True)
    if len(approx)==4:
        screen=approx
        break
cv2.drawContours(img,[screen],-1,(0,255,0),2)

cv2.imshow('edge',edge)
cv2.imshow('output_draw',img_resize(img,height=650))
cv2.waitKey(5)

-1

In [21]:
cv2.destroyAllWindows()

In [22]:
warped=four_point_transform(orig,screen.reshape(4,2)*ratio)
warped=cv2.cvtColor(warped,cv2.COLOR_BGR2GRAY)
thresh=threshold_local(warped,11,offset=10,method="gaussian")
warped=(warped>thresh).astype("uint8")*255

In [23]:
cv2.imshow('output_draw',img_resize(img,height=650))
cv2.imshow('Out_final',img_resize(warped,height=650))
cv2.waitKey(1)

-1

In [24]:
cv2.destroyAllWindows()